##Color by similarity

**How to:**
1.   Choose the wanted options below.
2.   Press "Runtime" -> "Run all".
3.   Press "Choose Files" and choose all the structures that you want to analyse (At least 3 structures).
4.   A dropdown menu will appear. Choose the structure that you want to be the reference and Press "SELECT".

In [1]:
!pip install pymol

In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()
condacolab.check()

In [ ]:
!mamba install -c conda-forge rdkit
!mamba install -c conda-forge pymol-open-source
!wget https://raw.githubusercontent.com/rdkit/rdkit/master/Docs/Book/data/cdk
!which pymol

In [ ]:
import subprocess
cmd = subprocess.Popen(["pymol", "-cKRQ"])

In [ ]:
!ps aux | grep pymol

In [ ]:
from pymol import cmd

In [3]:
cmd.load("../1nof.cif")

AttributeError: module '_distutils.cmd' has no attribute 'load'